In [2]:
from datetime import date
from datetime import datetime
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import A4
import pandas as pd
import numpy as np
import openpyxl
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import os

In [3]:
class Main:
    def main(self):
        self.url = "https://docs.google.com/spreadsheets/d/1HYjfEe3aCbufbqIXKs0Xz-gfoQNztGhCN1ivx0gZXnc/export?format = xlsx"    
        self.Clone_the_dataset_to_this_machine(self.url)
        self.data = self.creat_data("Student Gradebook.xlsx")
        self.data = self.Add_Month_column(self.data)
        return self.data
                
    def Clone_the_dataset_to_this_machine(self, url):
        a = requests.get(url)
        resp = requests.get(url)
        output = open('Student Gradebook.xlsx',  'wb')
        output.write(resp.content)
        output.close()
        
        
    def creat_data(self, file):
        wb = openpyxl.load_workbook(file) 
        Number_of_sheets = len(wb.sheetnames)
        for i in range(len(wb.sheetnames)):
            try:
                data = pd.concat([data,  pd.read_excel(file, sheet_name = i)])
            except:
                data = pd.read_excel(file, sheet_name = 0)
            #print(data.shape)
        return data

        
    def Add_Month_column(self, data):
        date_list = []
        day_list = []
        year_list = []
        a = 0
        for i in data["Date"]:
            try:
                date_list.append(i.strftime("%B"))
                day_list.append(i.strftime("%d"))
                year_list.append(i.strftime("%Y"))
                a  += 1
            except:
                try:
                    date_list.append(datetime.strptime(str(i), "%d/%m/%Y").strftime("%B"))
                    day_list.append(datetime.strptime(str(i), "%d/%m/%Y").strftime("%d"))
                    year_list.append(datetime.strptime(str(i), "%d/%m/%Y").strftime("%Y"))
                    a += 1
                except:
                    print("Interrupt in a line number : - ", a, "which is having a entry of:-", i)
                    break
        data["Date(only)"] = day_list
        data["Month"] = date_list
        data["Year"] = year_list
        return data

In [4]:
Call = Main()

In [5]:
data = Call.main()

In [6]:
data.head()

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Date(only),Month,Year
0,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,22,July,2019
1,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,22,July,2019
2,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,22,July,2019
3,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,22,July,2019
4,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,22,July,2019


In [7]:
data.groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()

,Student,Points
0,Kunal,373.5
1,Surabhi,370.0
2,Dipam,368.5
3,Ujjainee,366.0
4,Shakib,362.0
5,Purbita,361.0
6,Vishal,353.5
7,Sonali,349.5
8,Chandrima,349.0
9,Sharika,342.0


In [ ]:
def rank_of_the_student(self, name, month):
    working_data = self.data
    return str(5)

In [ ]:
def late_Submition_Ratio(self, name, month):
    working_data = self.data
    return str(0.5)

In [ ]:
def percentage_of_the_student(self, name, month):
    working_data = self.data
    return str(88)

In [ ]:
def percentile_of_the_student(self, name, month):
    working_data = self.data
    return str(95)

In [ ]:
def table_Content(self, name, month):
    working_data = self.data
    a = [["1Data Analytics", 30.0, 26.0, 23.0, 89], 
         ["2Introduction to", 30.0, 26.0, 23.0, 89], 
         ["3Data Analytics", 30.0, 26.0, 23.0, 89], 
         ["4Data Analytics", 30.0, 26.0, 23.0, 89], 
         ["5Introduction to", 30.0, 26.0, 23.0, 89]]
    return a

In [ ]:
def table_summary(self, name, month):
    working_data = self.data
    return ["TOTAL", 89, 26.0, 23.0, None]